In [3]:
#         Save the results to a new csv file.
""""This notebook contains steps to clean the retrieved tweets and extract features"""
# from langdetect.lang_detect_exception import LangDetectException
import csv
# from langdetect import detect
# with open("tweets_english.csv", "w", encoding='utf-8') as f:
#     f.write("date,user_name,user_id,text,reply_to,retweet_count,favorite_count,retweet_status,user_mentions,hashtags\n")
# with open('out_per_id2.csv', 'rt', encoding='utf-8') as f2:
#     old_csv = csv.reader(f2)
#     next(old_csv)
#     for row in old_csv:
#         #print(row)
#         text = row[3]
#         try:
#             if detect(text)=='en':
#                 with open("tweets_english.csv", "a", encoding='utf-8') as f:
#                     f.write(','.join(row)+"\n")
#         except LangDetectException:
#             print("\n")

In [22]:
import re
# The emoji unicode patterns below are taken from
#       https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
# However I am counting the emojis individually instead of extracting patterns with one or more emojis
# URL pattern and inspiration for mentions and hashtags are taken from
#   https://stackoverflow.com/questions/8376691/how-to-remove-hashtag-user-link-of-a-tweet-using-regular-expression
emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]", re.UNICODE)
hashtag = re.compile('(#[A-Za-z0-9]+)')
mention = re.compile('(@[A-Za-z0-9_]+)')
url = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)||(https//t.co/?)([A-Za-z0-9]+)')

In [23]:
#   Count hashtags and remove '#'
import csv
import pandas as pd
import numpy as np
df = pd.read_csv('tweets_english.csv', encoding='utf-8')
df = df.dropna()
users = df['user_name']
emoji_counts = np.zeros(len(users))
hashtag_counts = np.zeros(len(users))
has_url = np.empty(len(users), dtype='bool')
has_mention = np.empty(len(users), dtype='bool')
new_texts = np.empty(len(users), dtype='object')
for index, row in df.iterrows():
    # Find and count/report patterns
    text = row['text']
    emojis = emoji.findall(text)
    emoji_counts[index] = len(emojis) # Add emoji counts to array
    hashtags = hashtag.findall(text)
    hashtag_counts[index] = len(hashtags)
    match_url = re.search(url, text)
    match_mention = re.search(mention, text)
    if match_url:
        has_url[index] = True
    if match_mention:
        has_mention[index] = True
    # Remove patterns from text
    new_text = re.sub(emoji, '', text)
    new_text = re.sub('#', '', new_text)
    new_text = re.sub('RT', '', new_text)
    new_text = re.sub(':', '', new_text)
    new_text = re.sub(mention, '', new_text)
    new_text = re.sub(url, '', new_text)
    new_texts[index] = new_text
df['text'] = new_texts
df['emoji_count'] = emoji_counts
df['hashtag_count'] = hashtag_counts
df['has_mention'] = has_mention
df['has_url'] = has_url
df = df.dropna()
#df.to_csv('out2_cleaned.csv', index=False)
df.to_csv('tweets_english_cleaned.csv', index=False)



In [68]:
import numpy as np
df = pd.read_csv('tweets_english_cleaned.csv')
users = df.user_name.unique()
tweet_counts = np.zeros(len(users)) # How many tweets per user we have in the dataset
median_retweet_counts = np.zeros(len(users)) # Median retweet value for the user's tweets
mean_retweet_counts = np.zeros(len(users)) # Mean retweet value
sd_retweet_counts = np.zeros(len(users)) # Standard deviation of retweet count
original_tweet_counts = np.zeros(len(users)) # How many tweets are not retweets from others
original_medians = np.zeros(len(users)) # Median retweet count for original tweets
original_means = np.zeros(len(users)) # Mean retweet count for original tweets
original_sds = np.zeros(len(users)) # Standard deviation of retweet count for original tweets

i = 0
for user in users:
    user_rows = df.loc[df['user_name'] == user]
    retweets = user_rows['retweet_count']
    original_tweets = user_rows.loc[user_rows['retweet_status']==False]
    original_retweets = original_tweets['retweet_count']
    
    # How many tweets per user:
    number_tweets = len(user_rows)
    tweet_counts[i] = number_tweets
    number_originals = len(original_retweets)
    original_tweet_counts[i] = number_originals
    # Median retweet counts:
    median_retweet_count = retweets.median()
    median_retweet_counts[i] = median_retweet_count
    original_median = original_retweets.median()
    original_medians[i] = original_median
    # Mean retweet counts:
    mean_retweet_count = retweets.mean()
    mean_retweet_counts[i] = mean_retweet_count
    original_mean = original_retweets.mean()
    original_means[i] = original_mean
    # Standard deviations:
    general_sd = retweets.std()
    sd_retweet_counts[i] = general_sd
    original_sd = original_retweets.std()
    original_sds[i] = original_sd
    i = i+1
user_statistics = {'users' : users}
user_statistics = pd.DataFrame(user_statistics)
additional_data = {'tweet_counts' : tweet_counts,
                   'median_retweets' : median_retweet_counts,
                   'mean_retweets' : mean_retweet_counts,
                   'sd_retweets' : sd_retweet_counts,
                   'original_tweet_counts' : original_tweet_counts,
                   'original_median_retweets' : original_medians,
                   'original_mean_retweets' : original_means,
                   'original_sd_retweets' : original_sds}
df3 = pd.DataFrame(additional_data)
user_statistics = user_statistics.join(df3)
user_statistics.to_csv('tweets_english_user_statistics.csv', index=False)

0          0
1       3914
2          1
3         57
4      50345
       ...  
243        0
244      348
245        0
246        0
247     1327
Name: retweet_count, Length: 248, dtype: int64
248         0
249     21152
250         0
251         0
252         0
        ...  
1146    98132
1147    18573
1148    51146
1149        1
1150        0
Name: retweet_count, Length: 903, dtype: int64
1151    24
1152     2
1153     2
1154     0
1155     4
        ..
1957    10
1958     2
1959     0
1960    21
1961     0
Name: retweet_count, Length: 811, dtype: int64
1962     1
1963     0
1964     0
1965     0
1966    13
        ..
2655     6
2656    30
2657     1
2658     1
2659     1
Name: retweet_count, Length: 698, dtype: int64
2660        0
2661        0
2662     3401
2663        0
2664        0
        ...  
3286      206
3287    11629
3288        0
3289      171
3290        0
Name: retweet_count, Length: 631, dtype: int64
3291      754
3292       14
3293        0
3294      705
3295     4605
  

In [24]:
import pandas as pd
import numpy as np
df = pd.read_csv('tweets_english_cleaned.csv')
#df = pd.read_csv('tiny_no_emojis.csv')
tweets = df['text']
user_statistics = pd.read_csv('tweets_english_user_statistics.csv')
popularity = np.zeros(len(tweets), dtype='object')
for index, row in df.iterrows():
    user = row['user_name']
    user_statistic = user_statistics.loc[user_statistics['users']==user]
    user_statistic = user_statistic
    median_popularity = user_statistic.iloc[0]['median_retweets']
    this_popularity = row['retweet_count']
    if this_popularity<median_popularity:
        popularity[index] = 0
    elif this_popularity>median_popularity:
        popularity[index] = 1
    else:
        popularity[index] = 0.5
df['relative_popularity'] = popularity
#df.to_csv('tiny_relative_popularities.csv', index=False)
df.to_csv('tweets_english_relative_popularities.csv', index=False)



In [27]:
import pandas as pd
import numpy as np
df = pd.read_csv('tweets_english_relative_popularities.csv')
#df = pd.read_csv('tiny_relative_popularities.csv')
df['log_retweet_count'] = np.log(df['retweet_count'])
df.to_csv('tweets_english_all_features.csv',index=False)

In [29]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
df = pd.read_csv('tiny_relative_popularities.csv')
texts = df['text']
# Concatenate all tweets into a single string
all_tweets_text = texts.str.cat(sep=' ')
#print(all_tweets_text)

# Step 2: Tokenize (normalize as necessary)
tokenizer = TweetTokenizer()
tokens = tokenizer.tokenize(all_tweets_text)
print(tokens)
# Step 3: Get list of unique tokens
# (Step 4: Assign numeric value to each unique token?)

RT @auntiechristi: i cant change what i’ve done in the past but i can make sure i don’t make those same mistakes in the future RT @Abdulmir19: Bringing this back incase someone needed it https://t.co/FVM2PNQbvJ Wind blows the hardest the closer you get to the mountaintops RT @wxldflwrr: When I say I need a hug  this is what I mean RT @throwback_mr: "Always and Forever" 1977 https://t.co/BzEtk1JNRx RT @JimmyLou_: HAPPY CHARTER DAY TO THE ONLY CHAPTER THAT MATTERS !!!!!!  @RhoThetaPsiPhi   IF YOU DONT SAY THE “BLOODY” YOU AINT SAYIN… RT @landino887: Happy birthday to my little sis I love you twin RT @llourdess6: NEED MY BACK CRACKED LIKE A GLOW STICK RT @jonnyco_: Shawty will forever say “I got this” even with tears in her eyes. RT @aswadshiraz: STAY AT HOME IF YOU CAN T RUN LIKE THIS https://t.co/s47kJo7dDi RT @Kinglrg_: Women must really hate to see men at peace https://t.co/op3KtF2tyn RT @Iexorangecounty: im still fucking crying at this https://t.co/XuoMRHr16Z RT @QUEENNI16: This that